# Scientific Computing in Python: NumPy

In [ ]:
import numpy as np

## Introduction to NumPy

NumPy (short for Numerical Python) was created in 2005 by merging Numarray into Numeric. Since then, the open source NumPy library has evolved into an essential library for scientific computing in Python. It has become a building block of many other scientific libraries, such as SciPy, Scikit-learn, Pandas, and others.

What makes NumPy so useful is that it provides a convenient Python interface for working with multi-dimensional array data structures efficiently. The NumPy array data structure is also called `ndarray`, which is short for *n*-dimensional array.

In addition to being mostly implemented in C and using Python as a "glue language", the main reason why NumPy is so efficient for numerical computations is that NumPy arrays use contiguous blocks of memory that can be efficiently cached by CPI. In contrast, Python lists are arrays of pointers to objects in random locations in memory, which cannot be easily cached and come a more expensive memory look-up.

However, the computational efficiency and low-memory footprint come at a cost: NumPy arrays have a fixed size and are homogeneous, which means that elements must have the same type. Homogenous `ndarray` objects have the advantage of using a C execution backend and avoiding expensive type checks and other overheads of the Python API.

While adding and removing elements from the end of a Python list is very efficient, altering the size of a NumPy array is very expensive since it requires the creation of a new array & carrying over the contents of the old array that we want to expand or shrink.

Besides being more efficient for numerical computations that native Python code, NumPy can also be more elegant and readable due to vectorized operations and broadcasting, which are features that we will cover in this notebook.

In summary, today NumPy forms the basis of the scientific Python computing ecosystem.

### Motivation: NumPy is fast!

To motivate the use of NumPy, we take a look at a latency comparison with regular Python code. In particular, we are computing a factor dot product in Python (using lists) and compare it with NumPy's dot-product function.

Mathematically, the dot product between two vectors $\mathbf{x}$ and $\mathbf{w}$ can be written as follows:

$$z = \sum_{i}x_{i}w_{i} = x_{1} \times w_{1} + x_{2} \times w_{2} + \dots + x_{n} \times w_{n} = \mathbf{x^{\top}}\mathbf{w}$$

Let's start by implementing dot-product using plain Python:

In [ ]:
# Define the dot product function
def dot_product(x, w):
    z = 0.
    for i in range(len(x)):
        z += x[i] * w[i]
    return z

# Create sample vectors
a = [1., 2., 3.]
b = [4., 5., 6.]

product = dot_product(a, b)
print(f"The dot product between {a} and {b} is: {product}")

Let's us estimate the run time for two larger (1000-element) vectors using IPython's `%timeit` magic function:

In [ ]:
A = list(range(1000))
B = list(range(1000))

%timeit dot_product(A, B)

Next, we use the `dot` function offered by NumPy to compute the same product:

In [ ]:
# Transform the lists into NumPy arrays
A_arr = np.array(A)
B_arr = np.array(B)

# Estimate the time to compute the dot product
%timeit _ = A_arr.dot(B_arr)

As we can see, replacing Python's for-loop on normal lists with NumPy's dot function makes the computation of the vector dot product approximately **100 times faster**.

### N-dimensional Arrays

NumPy is built around the [ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) object, which is a high-performance multi-dimensional array data structure. 

Intuitively, we can think of a one-dimensional NumPy array as a data structure that represents a vector of elements, you may also think of it as a fixed-size Python list where all elements share the same data type. Similarly, we can think of a two-dimensional array as a data structure that represents a matrix or a Python list of lists.

While NumPy arrays can have up to 32 dimensions, we will focus on lower-dimensional arrays for the purpose of illustration of this introduction. 

Let us get started with NumPy by calling the `array` function to create a two-dimensional NumPy array that consists of two rows and three columns, from a Python list of lists:

In [ ]:
# Create a python list of lists
nums = [
    [1, 2, 3],
    [4, 5, 6]
]

# Use it to create a 2D numpy array
arr = np.array(nums)

# 2 rows & 3 columns
arr  

<img src="../static/numpy/imgs/numpy-intro/array_1.png" alt="Drawing" style="width: 300px;"/>

By default, NumPy infers the type of the array upon construction. Since we passed Python integers to the array, the `ndarray` object `arr` should be of type `int64` on a 64-bit machine, which we can confirm by accessing the `dtype` attribute:

In [ ]:
arr.dtype

If we want to construct NumPy arrays of different types, we can pass a value to the `dtype` argument of the `array` function. 

For example, we can use `np.int32` to create a 32-bit array. For a full list of supported data types, please refer to the official [NumPy documentation](https://numpy.org/doc/stable/user/basics.types.html). 

Once an array has been constructed, we can downcast or recast its type via the `astype` method as showing in the following examples:

In [ ]:
# Convert to 32-bit (int)
int32_arr = arr.astype(np.int32)
int32_arr

In [ ]:
# Convert to 32-bit (float)
float32_arr = arr.astype(np.float32)
float32_arr

In [ ]:
# Get the data type of the 32-bit-float array
float32_arr.dtype

To return the number of elements in an array, we can use the `size` attribute:

In [ ]:
# rows x cols
arr.size

We can also print the number of dimensions in our array using the `ndim` attribute:

In [ ]:
# axis 0: rows
# axis 1: cols
arr.ndim

What is always used by data scientists is the **`shape` attribute** that returns the number of dimensions and the coordinate size along each dimension:  

In [ ]:
# 2 rows and 3 columns
arr.shape

The returned `shape` is always a tuple; in the code example above, the two-dimensional `arr` object has two rows and three columns, `(2, 3)`, we can think of it as a matrix representation.

Conversely, the `shape` of a one-dimensional array only contains a single value:

In [ ]:
nums = [1., 2., 3.]
np.array(nums).shape

---

## NumPy Array Construction & Indexing

### Array Construction Routines

This sections provides a non-comprehensive list of array construction functions. 

These are simple yet useful NumPy functions used to initialize array with default values. Examples:

In [ ]:
# Create an multi-dimensional array of ones by providing the shape
np.ones(shape=(3, 4), dtype=np.int64)

In [ ]:
# Create a square matrix of zeros
np.zeros(shape=(3, 3))

We can use these functions to create arrays with arbitrary values by changing the default values. Example:

In [ ]:
np.zeros((3, 3)) + 99

Creating arrays of ones or zeros can be useful as placeholder arrays, in cases where we do not want to use the initial values for computations but want to fill it with other values right away.

In case we do not need the initial values, there is also `numpy.empty` which follows the same syntax as `numpy.ones` or `numpy.zeros`. However, instead of filling the array with a particular value, the `empty` function creates the array with non-sensical values from memory.

We can think of `zeros` as a function that creates the array via `empty` then sets all its values to `0`. In practice though, the difference in runtime between `zeros` and `empty` is not noticeable.

NumPy also comes with functions to create identity and diagonal matrices as `ndarray`s that can be useful in the context of linear algebra:

In [ ]:
# Create a 3x3 identity matrix
np.eye(3)

In [ ]:
# Create a diagonal matrix where we provide (only) the values at the diagonal
np.diag((1, 2, 3))

Lastly, we need to mention two very useful functions for creating sequences of numbers within a specified range, namely, `arange` and `linspace`.

NumPy's `arange` function follows the same syntax as Python's range objects: if two arguments are provided, the first argument represents the start value and the second argument defines the stop value of a half-open interval:

In [ ]:
np.arange(4, 10)

Notice that `arange` also performs type inference similar to the array function.

If we only provide a single function argument, the range object treats this number as the endpoint of the interval and starts at 0.

In [ ]:
np.arange(5)

Similar to Python's `range`, a third argument can be provided to define the step (the default step value is `1`). 

For example, we can obtain an array of all uneven values between one and ten as follows:

In [ ]:
np.arange(start=1., stop=11., step=.1)

The `linspace` function is especially useful if we want to create a particular number of evenly spaces values in a specified half-open interval:

In [ ]:
np.linspace(6., 15., num=33)

### Array Indexing

In this section, we will go over the basics of retrieving NumPy array elements via different indexing methods. Simple NumPy indexing and slicing works similar to Python lists, which we demonstrate as follows: 

In [ ]:
# Retrieve the first element of a one-dimensional array
arr = np.array([1, 2, 3])
arr[0]

Also, the same Python semantics apply to slicing operations. The following example shows to fetch the first two elements in `arr`:

In [ ]:
arr[0:2]  # or arr[:2]

If we work with arrays that have more than one dimension, we separate our indexing or slicing operations by commas as shown in the following examples:

In [ ]:
arr = np.array([
    [1, 2, 3],
    [4, 5, 6]
])
arr[0, -2]  # First rows, second from last column

In [ ]:
arr[-1, -1]  # Lower right

In [ ]:
arr[1, 1]  # Second row, second column

<img src="../static/numpy/imgs/numpy-intro/array_2.png" alt="Drawing" style="width: 300px;"/>

In [ ]:
arr[:, 0] # All rows, first column (e.g., the entire first column)

In [ ]:
arr[1, :]  # The entire second row

In [ ]:
arr[:, :2]  # The first two columns

---

## NumPy Array Math & Universal Functions

In the previous sections, we learned how to create NumPy arrays and how to access different elements in an array. It is about time we introduce one of the core features of NumPy that makes working with `ndarray` so efficient and convenient: **vectorization**.

While we typically use for-loops if we want to perform arithmetic operations on sequence-like objects, NumPy provides vectorized wrappers for performing element-wise operations implicitly via the so-called `ufuncs` (short for universal functions).

As of this writing, there are more than 60 `ufuncs` available in NumPy, `ufuncs` are implemented in compiled C code and are very fast and efficient compared to vanilla Python. 

In this section, we will take a look at the most commonly used `ufuncs`. We recommend to check out the [official documentation](https://numpy.org/doc/stable/reference/ufuncs.html#available-ufuncs) for a complete list.

To provide an example of a simple universal function for element-wise addition, consider the following example, where we add a scalar (here: $1$) to each element in a nested Python list:

In [ ]:
# Define a 2D list of lists
nums = [
    [1, 2, 3],
    [4, 5, 6]
]

# Increment each element by looping
rows, cols = len(nums), len(nums[0])
for row in range(rows):
    for col in range(cols):
        nums[row][col] += 1

# Check the list of lists values
nums

This for-loop approach is very verbose. We could achieve the same goal more elegantly using list comprehensions:

In [ ]:
nums = [[1, 2, 3], [4, 5, 6]]
[[v + 1 for v in row] for row in nums]

We can accomplish the same using NumPy's "add" ufunc for element-wise scalar addition as shown below: 

In [ ]:
arr = np.array([[1, 2, 3], [4, 5, 6]])
arr = np.add(arr, 1)  # Binary ufunc
arr

The ufuncs for basic arithmetic operations include `add`, `subtract`, `divide`, `multiply`, `power`, and `exp`.

Furthermore, NumPy uses operator overloading so that we can use the mathematical operators (`+`, `-`, `/`, `*`, and `**`) directly:

In [ ]:
# Verbose
np.add(arr, 1)

In [ ]:
# Or..
arr + 1

In [ ]:
# Verbose
np.power(arr, 2)

In [ ]:
# Or..
arr ** 2

Above, we have seen examples of binary ufuncs, which are ufuncs that take two arguments as an input. In addition, NumPy implements several useful *unary ufuncs*, such as `log`, `log10`, and `sqrt` (square root): 

In [ ]:
np.sqrt(arr)

Often, we want to compute the sum or product of array elements along a given axis. For this purpose, we can use a ufunc's `reduce` operation. 

By default, `reduce` applies an operation along the first axis (`axis=0`). In the case of a two-dimensional array, we can think of the first axis as the rows of a matrix. Thus, adding up elements along rows yields the column sums of that matrix as follows:

In [ ]:
# Create the array 
arr = np.array([[1, 2, 3], [4, 5, 6]])

# Reduce along the first dimension (rows)
np.add.reduce(arr, axis=0)

To compute the row sums (addition along the `column` dimension) of the array above, we can specify `axis=1`:

In [ ]:
np.add.reduce(arr, axis=1)

NumPy also provides shorthands for specific operations such as `product` and `sum`.

For example, `sum(axis=0)` is equivalent to `arr.add.reduce(axis=0)`:

In [ ]:
arr.sum(axis=0)

In [ ]:
arr.sum(axis=1)

<img src="../static/numpy/imgs/numpy-intro/ufunc.png" alt="Drawing" style="width: 600px;"/>

As a word of caution, keep in mind that `product` & `sum` compute the product or sum of the entire array if we do not specify an axis:

In [ ]:
arr.sum()

Other useful unary `ufuncs` are:
    
- `np.mean`: computes arithmetic mean or average.
- `np.std`: computes the standard deviation.
- `np.var`: computes variance.
- `np.sort`: sorts an array.
- `np.argsort`: returns indices that would sort an array.
- `np.min`: returns the minimum value of an array.
- `np.max`: returns the maximum value of an array.
- `np.argmin`: returns the index of the minimum value.
- `np.argmax`: returns the index of the maximum value.
- `np.array_equal`: checks if two arrays have the same shape and elements.

---

## Array Broadcasting

A topic we glanced over in the previous section is broadcasting. Boradcasting allows us to perform vectorized operations between two arrays even if their dimensions do not match by creating implicit multi-dimensional grids.

We already learnt about `ufuncs` in the previous section where we performed element-wise addition between a scalar and a multi-dimensional array, which is just one example of broadcasting:

<img src="../static/numpy/imgs/numpy-intro/broadcasting-1.png" alt="Drawing" style="width: 500px;"/>

Naturally, we can also perform element-wise operations between arrays of equal dimensions.

In contrast to what we are used from linear algebra, we can also add arrays of different shapes. In the example above, we added a 0-dimensional number to a one-dimensional array, in this case, NumPy creates an implicit multi-dimensional grid from the 0-dimensional array:

In [ ]:
arr = np.array([1, 2, 3])
arr + 1

In [ ]:
# Equivalent to..
arr + np.array([1, 1, 1])

We can do the same using 1-and-2 dimensional arrays. Example:

<img src="../static/numpy/imgs/numpy-intro/broadcasting-2.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
arr2 = np.array([
    [4, 5, 6],
    [7, 8, 9]
])
arr2 + arr

---

## NumPy Advanced Indexing: Memory Views & Copies

In the previous sections, we have used basic indexing and slicing routines. It is important to note that basic integer-based indecing and slicing create so-called **views** of NumPy arrays in memory. Working with views can be highly desirable since it avoids making unnecessary copies of arrays to save memory resources.

To illustrate the concept of memory views, let us walk through a simple example where we access the first row in an array, assign it to a vairbale, modify the variable, and finally check the original array:

In [ ]:
# Create a 2D array
arr = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

# Get the first row
row0 = arr[0]
row0

In [ ]:
# Change the first row variable
row0 += 99
row0

As expected, the first row has been modified. How about the original array:

In [ ]:
arr  # Modified too!

As shown, changing the vlaue of `row0` also affected the original array. The reason for this is that `arr[0]` created a view of the first row in `arr`, and its elements where then incremented by `99`.

The same concept applies to the slicing operator:

In [ ]:
# Create a 2D array
arr = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

# Get the second row's 
arr_slice = arr[:, 1:]
arr_slice -= 99

# Check the original array
arr

When we work with NumPy arrays, we should always be aware that **slicing creates views**, and sometimes this property is desirable to speed up our code and avoid creating unnecessary copies in memory. 

However, in certain scenarios we want to force a copy of an array; we can do this via the `copy` method as shown:

In [ ]:
# Create a 2D array
arr = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

# Copy the first row
row0 = arr[0].copy()
row0 += 1

print(row0)
print(arr)

### Fancy indexing

In addition to basic single-integer indexing and slciing operations, NumPy supports advanced indexing routines called *fancy indexing*. 

Via fancy indexing, we can use tuple or list objects of non-contiguous integer indices to return the desired array elements. Since fancy indexing can be performed with non-contiguous sequences, it cannot return a view (a contiguous slice from memory). Thus, fancy indexing always returns a copy of an array. 

Examples:

In [ ]:
# Create a 2D array
arr = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

# Get the first and last columns
arr_slice = arr[:, [0, 2]]
arr_slice += 99

# The slice changed
print(arr_slice)

# .. but the original array remains the same
print(arr)

### Boolean Masks for Indexing

We can also use boolean masks for indexing, that is, arrays for `True` and `False` values.

Consider the following example, where we return all values in the array that are greater than `3`:

In [ ]:
# Create a 2D array
arr = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

# Create the mask
mask = arr > 3
mask

Using the mask, we can select the elements of interest:

In [ ]:
arr[mask]

We can also chain different selection criteria using the logical "&" (e.g., 'and') or "|" (e.g., 'or').

The example below demonstrates how we can select array elements that are greater than 3 and divisible by 2:

In [ ]:
mask = (arr > 3) & (arr % 2 == 0)
arr[mask]

Note that indexing using boolean arrays is also considered "fancy indexing" and thus returns a copy of the original array.

---

## Random Number Generators

In machine learning and deep learning, we often have to generate arrays of random numbers. For example, the initial values of our model parameters before optimization.

`NumPy` has a `random` subpackage to create random numbers and samples from a variety of distributions conveniently. We encourage you to browse through the more comprehensive [numpy.random documentation](https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html) for a complete list of functions for random sampling.

To provide a brief overview of the pseudo-random number generators that we will use most commonly, let's start with drawing a random sample from a uniform distribution:

In [ ]:
# Set the seed or freeze the randomness
np.random.seed(123)

# Generate a random number from the uniform distribution of shape (3,)
np.random.rand(3)

In the code snippet above, we first seeded NumPy's random generator. Then, we drew three random samples from the uniform distribution via `random.rand` in the half-open interval `[0,1)`.

We highly recommendthe seeding step in practical appplications as well as in research projects, since it ensures that our results are reproducible. If we run our code sequentially, it should be sufficient to seed the random number generator only once at the beginning of the script to enforce reproducible outcomes between different runs.

In addition, it is often useful to create separate `RandomState` objects for various parts of our code so that we can test methods of functions reliably in unit tests. It can also be useful if we run our code in non-sequential order (e.g., interactive sessions or Jupyter notebook environments).

The example below shows how we can use a `RandomState` object to create the same results that we obtained via `np.random.rand` in the previous code snippet:

In [ ]:
random_generator = np.random.RandomState(seed=123)
random_generator.rand(3)

---

## Reshaping NumPy Arrays

In practice, we often run into situations where existing arrays do not have the *right* shape to perform certain computations. As we might remember from the beginning of this notebook, the size of NumPy arrays is fixed. Fortunately, this does not mean that we have to create new arrays and copy values from the old array to the new one if we want arrays of different shapes. 

In summary, the size is fixed, by the shape is not. NumPy provides the `reshape` method that allows us to obtain a view of an array with a different shape.

For example, we can reshape a one-dimensional array into a two-dimensional one using `reshape` as follows:

In [ ]:
arr = np.array(range(1, 7))
arr_view = arr.reshape(2, 3)
arr_view

We can verify that the `arr_view` is indeed a view of `arr` by calling the `may_share_memory` function:

In [ ]:
np.may_share_memory(arr, arr_view)

When we specify the desired elements along each axis, we need to make sure that the reshaped array has the same number of elements as the original one. 

However, we do not need to specify the number of elements in all axis, NumPy is smart enough to figure out how many elements to put along an axis. if only one axis is unspecified (by using the placeholder `-1`):

In [ ]:
arr.reshape(-1, 2)  # We ask NumPy to figure out the number of rows, but we want two columns!

Consequently, we can use `-1` to flatten an array:

In [ ]:
# Create a 2D array
arr = np.array([
    [1, 2, 3],
    [4, 5, 6]
])

# Flatten
arr.reshape(-1)

Other methods for flattening arrays exist, namely `flatten`, which created a copy of the array, and `ravel`, which creates a memory view like `reshape`: 

In [ ]:
np.may_share_memory(arr, arr.flatten())

In [ ]:
np.may_share_memory(arr, arr.ravel())

Sometimes, we are interested in merging different arrays. Unfortunately, there is no way to do this without a new array, since NumPy arrays have a fixed size.

While combining arrays should b avoided if possible, it is sometimes necessary. To combine two or more array objects, we can use NumPy's `concatenate` function as shown in the following example: 

In [ ]:
arr = np.array([1, 2, 3])

# Stack arr with itself along the first dimension (and the only one)
np.concatenate([arr, arr])

We can add a dimension to the original array and stack by it:

In [ ]:
arr = arr.reshape(1, 3)
np.concatenate([arr, arr], axis=0)

In [ ]:
# Stack along the second dimension (columns)
np.concatenate([arr, arr], axis=1)

---

## NumPy Comparison Operators & Masks

Boolean masks are `bool`-type arrays that have the same shape as a certain target array. For example, using comparison operators (such as `<`, `>`, `<=`, and `>=`), we can create a Boolean mask of that array which consists of `True` and `False` elements depending on whether a condition is met:

In [ ]:
arr = np.array(range(1, 6))
mask = arr > 2
mask

Once we create such a mask, we can use it to select certain entries from the original array:

In [ ]:
arr[mask]

Beyond the selection of elements from an array, boolean masks can also come in handy when we want to count how many elements in an array meet a certain condition:

In [ ]:
mask.sum()

A related, useful function to assign values to specific elements in an array is the `np.where` function.

Example:

In [ ]:
arr = np.array(range(1, 6))

# Set non-masked to zero
np.where(arr > 2, arr, 0)

There are also the so-called bit-wise operators that we can use to specify more complex selection criteria:

In [ ]:
arr = np.array(range(1, 6))
mask = arr > 2
arr[mask] = 1
arr[~mask] = 0
arr

The `~` operator in the example above is one of the logical operators in NumPy:

- And: `&`.
- Or: `|`.
- XOR: `^`.
- Not: `~`.

These logical operators allow us to chain an arbitrary number of conditions to create even more "complex" Boolean masks.

For example, using the "Or" operator, we can select all elements that are greater than 3 or smaller than 2 as follows:

In [ ]:
arr = np.array(range(1, 6))
arr[(arr > 3) | (arr < 2)]

In [ ]:
# We also also negative boolean expressions
arr[~((arr > 3) | (arr < 2))]

## Linear Algebra

Intuitively, we can think of one-dimensional NumPy arrays as data structures that represent row vectors:

In [ ]:
row_vect = np.array(range(1, 4))
row_vect

We can reshape the original vector to create column vectors:

In [ ]:
# 3 rows, 1 column
row_vect.reshape(3, 1)

Instead of reshaping a one-dimensional array into a 2-dimensional one, we can simply add a new axis:

In [ ]:
row_vect[:, None]

All three approaches listed above yield the same result (they create views and not copies of the orignal array).

We can think of a column vector as a matrix consisting only of one column. 

To perform matrix multiplication between matrices, we learnt that the number of columns of the left matrix must match the number of rows of the right matrix.

In NumPy, we can perform matrix multiplication via the `matmul` function:

In [ ]:
M = np.array([[1, 2, 3], 
              [4, 5, 6]])

np.matmul(M, row_vect[:, None])

<img src="../static/numpy/imgs/numpy-intro/matmul.png" alt="Drawing" style="width: 300px;"/>

However, if we are working with matrices and vectors, NumPy can be quite forgiving if the dimensions of matrices and one-dimensional arrays do not match exactly (thanks to broadcasting).

The following example yields the same result as the matrix-column vector multiplication, except that it returns a one-dimensional array instead of a two-dimensional one:

In [ ]:
M.shape, row_vect.shape

In [ ]:
np.matmul(M, row_vect)

Similarly, we can compute the dot-product between two vectors:

In [ ]:
np.matmul(row_vect, row_vect)

In [ ]:
# Or using `.dot()`...
row_vect.dot(row_vect)

Note that an even more convenient way for executing `np.dot` is using the `@` symbol with NumPy arrays:

In [ ]:
M @ row_vect

Similar to the examples above, we can use `matmul` or `dot` to multiply two matrices. In this context, NumPy arrays have a handy `transpose` method to transpose matrices if necessary:

In [ ]:
# Verbose
matrix = np.array([[1, 2, 3], 
                   [4, 5, 6]])
matrix.transpose()

In [ ]:
# better...
matrix.T

<img src="../static/numpy/imgs/numpy-intro/transpose.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
np.dot(matrix, matrix.T)

<img src="../static/numpy/imgs/numpy-intro/matmatmul.png" alt="Drawing" style="width: 500px;"/>

While this section demonstrates some of the basic linear algebra operations carried out on NumPy arrays that we use in practice, you can find an additional function in the docs of NumPy's submodule for linear algebra: [`numpy.linalg`](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html).

If you want to perform a particular linear algebra routine that is not implemented in NumPy, it is worth consulting the [`scipy.linalg` documentation](https://docs.scipy.org/doc/scipy/reference/linalg.html). SciPy is a library for scientific computing built on top of NumPy.

## References

NumPy and Matplotlib reference material:

- [The official NumPy documentation](https://docs.scipy.org/doc/numpy/reference/index.html)
- [The official Matplotlib Gallery](https://matplotlib.org/gallery/index.html)
- [The official Matplotlib Tutorials](https://matplotlib.org/tutorials/index.html)


Optional references books for using NumPy and SciPy:

- Rougier, N.P., 2016. [From Python to NumPy](http://www.labri.fr/perso/nrougier/from-python-to-numpy/).
- Oliphant, T.E., 2015. [A Guide to NumPy: 2nd Edition](https://www.amazon.com/Guide-NumPy-Travis-Oliphant-PhD/dp/151730007X). USA: Travis Oliphant, independent publishing.
- Varoquaux, G., Gouillart, E., Vahtras, O., Haenel, V., Rougier, N.P., Gommers, R., Pedregosa, F., Jędrzejewski-Szmek, Z., Virtanen, P., Combelles, C. and Pinte, D., 2015. [SciPy Lecture Notes](http://www.scipy-lectures.org/intro/numpy/index.html).
- Harris, C.R., Millman, K.J., van der Walt, S.J. et al. [Array Programming with NumPy](https://www.nature.com/articles/s41586-020-2649-2). Nature 585, 357–362 (2020). 

## Credits

- [Original post](https://sebastianraschka.com/blog/2020/numpy-intro.html).

---